# The Beatles Chords

## Inspiration

This project was inspired by Hooktheory.
https://www.hooktheory.com/theorytab/common-chord-progressions
https://www.hooktheory.com/trends

These are some of the questions this project hopes to shed light on:
* What chords are most popular?
* How often do these chords show up in songs
* What tends to be the next chord?

The data was scraped from Ultimate Guitar.
https://www.ultimate-guitar.com/artist/the_beatles_1916?filter=chords
To demonstrate proof of concept, only The Beatles tabs were scraped and the chords in their songs were analyzed.



In [4]:
# parse through the csv file and create a data structure
# the data should include counts for the following:
# 1. total count of each chord and how often they are used
# 2. the number of songs each chord is used in
# 3. the count of what chords come after each

import csv

with open('the_beatles_chords.csv', mode='r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    # making dict removes duplicates
    chords_dict = {rows[0]:rows[1] for rows in reader}

# print(chords_dict)

data = {
    'total_count': {},
    'song_count': {},
    'next_chord': {}
}

for chord_str in chords_dict.values():
    chords = chord_str.split(',')
    unique_chords = list(set(chords))
    
    for chord in unique_chords:
        if chord in data['song_count']:
            data['song_count'][chord] += 1
        else:
            data['song_count'][chord] = 1
    
    
    for i, chord in enumerate(chords):
        if chord in data['total_count']:
            data['total_count'][chord] += 1
        else:
            data['total_count'][chord] = 1
        
        if i != len(chords)-1:
            next_chord = chords[i+1]
            if chord in data['next_chord']:
                if next_chord in data['next_chord'][chord]:
                    data['next_chord'][chord][next_chord] += 1
                else:
                    data['next_chord'][chord][next_chord] = 1
            else:
                data['next_chord'][chord] = {
                    next_chord: 1
                }
            
# print(data)
            

## Total chord counts in all songs

In [5]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet

total_count_chords = []
total_count_counts = []

for chord, count in data['total_count'].items():
    total_count_chords.append(chord)
    total_count_counts.append(count)
    
sorted_chords = sorted(total_count_chords, key=lambda c: total_count_counts[total_count_chords.index(c)])

source = ColumnDataSource(data=dict(chords=total_count_chords, counts=total_count_counts))

p = figure(y_range=sorted_chords,
           title="Total Chord Counts",
           toolbar_location=None,
           tools="",
           height=5000,
           sizing_mode="scale_width")
p.hbar(y='chords', right='counts', height=0.9, source=source)

labels = LabelSet(x='counts', y='chords', text='counts', level='glyph',
        x_offset=10, y_offset=-6, source=source, render_mode='canvas', text_font_size='12px')
p.add_layout(labels)

show(p)


## Counts of songs each chord appears in

In [6]:
song_count_chords = []
song_count_counts = []

for chord, count in data['song_count'].items():
    song_count_chords.append(chord)
    song_count_counts.append(count)
    
sorted_chords = sorted(song_count_chords, key=lambda c: song_count_counts[song_count_chords.index(c)])

source = ColumnDataSource(data=dict(chords=song_count_chords, counts=song_count_counts))

p = figure(y_range=sorted_chords,
           title="Chord Song Counts",
           toolbar_location=None,
           tools="",
           height=5000,
           sizing_mode="scale_width")
p.hbar(y='chords', right='counts', height=0.9, source=source)

labels = LabelSet(x='counts', y='chords', text='counts', level='glyph',
        x_offset=10, y_offset=-6, source=source, render_mode='canvas', text_font_size='12px')
p.add_layout(labels)

show(p)

## Analysis of next chords

In [10]:
from bokeh.models import CustomJS, DataSource, Dropdown
from bokeh.layouts import column

menu = [(chord, chord) for chord in data['total_count'].keys()]
dropdown = Dropdown(label="Chord", menu=menu)

chord_type = DataSource(name='')
source = ColumnDataSource(data=dict(chords=[], counts=[]))
callback = CustomJS(args=dict(source=source, type=chord_type, next_chord_data=data['next_chord']), code="""
        type.name = cb_obj.value;

        let data = source.data;
        let nextChords = next_chord_data[type.name]

        data['chords'] = Object.keys(nextChords);
        data['counts'] = Object.values(nextChords);

        source.change.emit();
    """)

dropdown.js_on_change('value', callback)

p = figure(y_range=sorted_chords,
           height=5000,
           toolbar_location=None,
           tools="",
           sizing_mode="scale_width")
p.hbar(y='chords', right='counts', height=0.9, source=source)

labels = LabelSet(x='counts', y='chords', text='counts', level='glyph',
        x_offset=10, y_offset=-6, source=source, render_mode='canvas', text_font_size='12px')
p.add_layout(labels)


show(column(dropdown, p))